In [37]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [38]:
df = pd.read_csv('../data/project_dataset2.csv')
print("Rows, columns:", df.shape)


Rows, columns: (1074, 99)


In [39]:
df.head()

,match_id,year,date,stage,group_name,home_team,away_team,home_team_norm,away_team_norm,home_goals,...,match_time,stadium_id,home_team_code,away_team_code,score,home_score_margin,away_score_margin,home_team_win,away_team_win,draw
0,1.0,1930,1930-07-13 00:00:00+00:00,Group stage,Group 1,France,Mexico,france,mexico,4.0,...,15:00,S-193,FRA,MEX,41,3.0,-3.0,1.0,0.0,0.0
1,2.0,1930,1930-07-13 00:00:00+00:00,Group stage,Group 4,United States,Belgium,united states,belgium,3.0,...,15:00,S-192,USA,BEL,30,3.0,-3.0,1.0,0.0,0.0
2,4.0,1930,1930-07-14 00:00:00+00:00,Group stage,Group 3,Romania,Peru,romania,peru,3.0,...,14:50,S-193,ROU,PER,31,2.0,-2.0,1.0,0.0,0.0
3,3.0,1930,1930-07-14 00:00:00+00:00,Group stage,Group 2,Yugoslavia,Brazil,serbia,brazil,2.0,...,12:45,S-192,YUG,BRA,21,1.0,-1.0,1.0,0.0,0.0
4,5.0,1930,1930-07-15 00:00:00+00:00,Group stage,Group 1,Argentina,France,argentina,france,1.0,...,16:00,S-192,ARG,FRA,10,1.0,-1.0,1.0,0.0,0.0


In [40]:
# Standardize important fields

# dates and numeric parsing
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

numeric_cols = [
    "home_goals",
    "away_goals",
    "home_xg",
    "away_xg",
    "attendance",
    "home_penalties",
    "away_penalties",
    "home_score_margin",
    "away_score_margin",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# stage normalization
if "stage_clean" in df.columns:
    df["stage_norm"] = df["stage_clean"]
else:
    df["stage_norm"] = df["stage"].astype(str).str.strip()

stage_map = {
    "Group Stage": "Group stage",
    "Group stage": "Group stage",
    "First group stage": "Group stage",
    "First round": "Group stage",
    "Second group stage": "Second group stage",
    "Round of 16": "Round of 16",
    "Quarter-finals": "Quarter-finals",
    "Semi-finals": "Semi-finals",
    "Final": "Final",
    "Third place": "Third place",
}

df["stage_norm"] = df["stage_norm"].replace(stage_map)

# score dash cleanup
for col in ["score", "score_penalties", "score_penalties_d7new"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace("", "-", regex=False)
        df.loc[df[col] == "nan", col] = np.nan

print("Stage values (top 10):")
print(df["stage_norm"].value_counts().head(10))

Stage values (top 10):
stage_norm
Group stage           748
Round of 16            97
Quarter-finals         66
Semi-finals            36
Second group stage     36
Final                  21
Third place            19
Round of 32            16
round of 16             8
group stage             6
Name: count, dtype: int64


In [41]:
df

,match_id,year,date,stage,group_name,home_team,away_team,home_team_norm,away_team_norm,home_goals,...,stadium_id,home_team_code,away_team_code,score,home_score_margin,away_score_margin,home_team_win,away_team_win,draw,stage_norm
0,1.0,1930,1930-07-13 00:00:00+00:00,Group stage,Group 1,France,Mexico,france,mexico,4.0,...,S-193,FRA,MEX,4-1,3.0,-3.0,1.0,0.0,0.0,Group stage
1,2.0,1930,1930-07-13 00:00:00+00:00,Group stage,Group 4,United States,Belgium,united states,belgium,3.0,...,S-192,USA,BEL,3-0,3.0,-3.0,1.0,0.0,0.0,Group stage
2,4.0,1930,1930-07-14 00:00:00+00:00,Group stage,Group 3,Romania,Peru,romania,peru,3.0,...,S-193,ROU,PER,3-1,2.0,-2.0,1.0,0.0,0.0,Group stage
3,3.0,1930,1930-07-14 00:00:00+00:00,Group stage,Group 2,Yugoslavia,Brazil,serbia,brazil,2.0,...,S-192,YUG,BRA,2-1,1.0,-1.0,1.0,0.0,0.0,Group stage
4,5.0,1930,1930-07-15 00:00:00+00:00,Group stage,Group 1,Argentina,France,argentina,france,1.0,...,S-192,ARG,FRA,1-0,1.0,-1.0,1.0,0.0,0.0,Group stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,NaN,2026,2026-07-12 02:00:00+00:00,Quarterfinals,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quarterfinals
1070,NaN,2026,2026-07-14 20:00:00+00:00,Semifinals,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Semifinals
1071,NaN,2026,2026-07-15 19:00:00+00:00,Semifinals,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Semifinals
1072,NaN,2026,2026-07-18 21:00:00+00:00,Third Place Playoff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Third Place Playoff


In [42]:
# Cleanup: date-only, numeric ints, remove win_conditions, and rename column names

# date to date-only (no time)
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date

# numeric columns to integers (nullable)
numeric_cols = [
    "match_id",
    "home_goals",
    "away_goals",
    "home_xg",
    "away_xg",
    "attendance",
    "home_penalties",
    "away_penalties",
    "home_score_margin",
    "away_score_margin",
    "home_team_win",
    "away_team_win",
    "draw",
    "extra_time",
    "penalty_shootout",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").round().astype("Int64")

# delete win_conditions
if "win_conditions" in df.columns:
    df = df.drop(columns=["win_conditions"])

# explicit renames (fill target if exists, then drop source)
explicit_map = {
    "stadium_name_d7": "stadium_name",
    "city_name_d7": "city_name",
    "host_country_d7": "host_country",
    "home_team_id_d7": "home_team_id",
    "away_team_id_d7": "away_team_id",
}
for src, tgt in explicit_map.items():
    if src in df.columns:
        if tgt in df.columns:
            df[tgt] = df[tgt].fillna(df[src])
            df = df.drop(columns=[src])
        else:
            df = df.rename(columns={src: tgt})


print("Columns after cleanup:")
print(df.columns.tolist())

Columns after cleanup:
['match_id', 'year', 'date', 'stage', 'group_name', 'home_team', 'away_team', 'home_team_norm', 'away_team_norm', 'home_goals', 'away_goals', 'home_xg', 'away_xg', 'attendance', 'stadium', 'city', 'host_country', 'referee', 'host_team', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_penalties', 'away_penalties', 'result', 'notes', 'dup_key', 'tournament_id_x', 'tournament_name_x', 'd7_match_id_x', 'match_name_x', 'group_name_d7new', 'group_stage_x', 'knockout_stage_x', 'replayed_x', 'replay_x', 'match_time_x', 'stadium_id_x', 'stadium_name_d7_x', 'city_name_d7_x', 'host_country_d7_x', 'home_team_id_d7_x', 'home_team_code_x', 'away_team_id_d7_x', 'away_team_code_x', 'score_x', 'home_score_margin_x', 'away_score_margin_x', 'extra_time_d7new', 'penalty_shootout_d7new', 'score_penalties_d7new', 'home_penalties_d7new', 'away_penalties_d7new', 'result_d7new', 'home_team_win_x', 'away_team_win_x', 'draw_x', 'tournament_id_y', 'tournament_name_y', 'd7_match_i

In [43]:
# remove any remaining "d7" mentions where no collision occurs
rename_map = {}
for col in df.columns:
    if "d7new" in col or "_d7" in col:
        new_col = col.replace("_d7new", "").replace("_d7", "")
        if new_col not in df.columns:
            rename_map[col] = new_col

if rename_map:
    df = df.rename(columns=rename_map)

print("Columns after cleanup:")
print(df.columns.tolist())

Columns after cleanup:
['match_id', 'year', 'date', 'stage', 'group_name', 'home_team', 'away_team', 'home_team_norm', 'away_team_norm', 'home_goals', 'away_goals', 'home_xg', 'away_xg', 'attendance', 'stadium', 'city', 'host_country', 'referee', 'host_team', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_penalties', 'away_penalties', 'result', 'notes', 'dup_key', 'tournament_id_x', 'tournament_name_x', 'd7_match_id_x', 'match_name_x', 'group_name_d7new', 'group_stage_x', 'knockout_stage_x', 'replayed_x', 'replay_x', 'match_time_x', 'stadium_id_x', 'stadium_name_x', 'city_name_x', 'host_country_x', 'home_team_id_x', 'home_team_code_x', 'away_team_id_x', 'away_team_code_x', 'score_x', 'home_score_margin_x', 'away_score_margin_x', 'extra_time_d7new', 'penalty_shootout_d7new', 'score_penalties_d7new', 'home_penalties_d7new', 'away_penalties_d7new', 'result_d7new', 'home_team_win_x', 'away_team_win_x', 'draw_x', 'tournament_id_y', 'tournament_name_y', 'd7_match_id_y', 'match_na

In [44]:
# Remove column
if "match_id" in df.columns:
    df = df.drop(columns=["match_id", "dup_key"])

In [45]:
# Fill missing stadium, city, host_country with "Not specified"
for col in ["stadium", "city", "host_country"]:
    if col in df.columns:
        df[col] = df[col].fillna("Not specified")

In [ ]:
# Rename tournament_id_x -> tournament_id and fill missing with WC-<year>
if "tournament_id_x" in df.columns:
    df = df.rename(columns={"tournament_id_x": "tournament_id"})

if "tournament_id" in df.columns and "year" in df.columns:
    df["tournament_id"] = df["tournament_id"].fillna("WC-" + df["year"].astype(str))

In [47]:
if "tournament_name_x" in df.columns:
    df = df.rename(columns={"tournament_name_x": "tournament_name"})

if "tournament_name" in df.columns and "year" in df.columns:
    df["tournament_name"] = df["tournament_name"].fillna(df["year"].astype(str) + " FIFA World Cup")

In [48]:
# Delete d7_match_id_x column
if "d7_match_id_x" in df.columns:
    df = df.drop(columns=["d7_match_id_x"])

In [49]:
# Rename match_name_x -> match_name and fill missing as "home_team v away_team"
if "match_name_x" in df.columns:
    df = df.rename(columns={"match_name_x": "match_name"})

if "match_name" in df.columns and "home_team" in df.columns and "away_team" in df.columns:
    df["match_name"] = df["match_name"].fillna(df["home_team"].astype(str) + " v " + df["away_team"].astype(str))

In [50]:
# Delete columns
for col in ["group_stage_x", "knockout_stage_x", "replayed_x"]:
    if col in df.columns:
        df = df.drop(columns=[col])

In [51]:
# Remove all "_x" suffixes in column names
rename_x = {col: col[:-2] for col in df.columns if col.endswith("_x") and col[:-2] not in df.columns}
if rename_x:
    df = df.rename(columns=rename_x)

In [52]:
# Remove all "_d7new" in column names
rename_d7new = {col: col.replace("_d7new", "") for col in df.columns if "_d7new" in col and col.replace("_d7new", "") not in df.columns}
if rename_d7new:
    df = df.rename(columns=rename_d7new)

In [53]:
# Delete specific columns requested
cols_to_drop = [
    "home_score_margin_x",
    "away_score_margin_x",
    "extra_time_d7new",
    "penalty_shootout_d7new",
    "score_penalties_d7new",
    "home_penalties_d7new",
    "away_penalties_d7new",
    "result_d7new",
    "home_team_win_x",
    "draw_x",
]
for col in cols_to_drop:
    if col in df.columns:
        df = df.drop(columns=[col])

In [54]:
print(df.columns.tolist())

['year', 'date', 'stage', 'group_name', 'home_team', 'away_team', 'home_team_norm', 'away_team_norm', 'home_goals', 'away_goals', 'home_xg', 'away_xg', 'attendance', 'stadium', 'city', 'host_country', 'referee', 'host_team', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_penalties', 'away_penalties', 'result', 'notes', 'tournament_id', 'tournament_name', 'match_name', 'group_name_d7new', 'replay_x', 'match_time_x', 'stadium_id_x', 'stadium_name', 'city_name', 'host_country_x', 'home_team_id', 'home_team_code_x', 'away_team_id', 'away_team_code_x', 'score_x', 'away_team_win_x', 'tournament_id_y', 'tournament_name_y', 'd7_match_id_y', 'match_name_y', 'group_stage_y', 'knockout_stage_y', 'replayed_y', 'replay_y', 'match_time_y', 'stadium_id_y', 'stadium_name_y', 'city_name_y', 'host_country_y', 'home_team_id_y', 'home_team_code_y', 'away_team_id_y', 'away_team_code_y', 'score_y', 'home_score_margin_y', 'away_score_margin_y', 'home_team_win_y', 'away_team_win_y', 'draw_y', 'tou

In [55]:
# Reorder columns so similar fields are grouped

preferred_order = [
    # tournament identifiers
    "year",
    "date",
    "tournament_id",
    "tournament_name",
    # match identifiers
    "match_name",
    "stage",
    "stage_norm",
    "group_name",
    "group_name_d7new",
    "group_stage",
    "knockout_stage",
    "replayed",
    "replay",
    # teams
    "home_team",
    "away_team",
    "home_team_norm",
    "away_team_norm",
    "home_team_id",
    "away_team_id",
    "home_team_code",
    "away_team_code",
    # score & outcome
    "home_goals",
    "away_goals",
    "score",
    "home_score_margin",
    "away_score_margin",
    "result",
    "home_team_win",
    "away_team_win",
    "draw",
    "extra_time",
    "penalty_shootout",
    "score_penalties",
    "home_penalties",
    "away_penalties",
    # xg
    "home_xg",
    "away_xg",
    # venue
    "stadium",
    "stadium_id",
    "stadium_name",
    "city",
    "city_name",
    "host_country",
    "host_team",
    "attendance",
    "match_time",
    # officials & notes
    "referee",
    "notes",
]

# keep only columns that exist, append any remaining columns at the end
ordered_cols = [c for c in preferred_order if c in df.columns]
remaining_cols = [c for c in df.columns if c not in ordered_cols]

df = df[ordered_cols + remaining_cols]

print("Reordered columns:")
print(df.columns.tolist())

Reordered columns:
['year', 'date', 'tournament_id', 'tournament_id', 'tournament_name', 'tournament_name', 'match_name', 'match_name', 'stage', 'stage_norm', 'group_name', 'group_name_d7new', 'group_stage', 'knockout_stage', 'replayed', 'replay', 'home_team', 'away_team', 'home_team_norm', 'away_team_norm', 'home_team_id', 'away_team_id', 'home_team_code', 'away_team_code', 'home_goals', 'away_goals', 'score', 'home_score_margin', 'away_score_margin', 'result', 'home_team_win', 'away_team_win', 'draw', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_penalties', 'away_penalties', 'home_xg', 'away_xg', 'stadium', 'stadium_id', 'stadium_name', 'city', 'city_name', 'host_country', 'host_team', 'attendance', 'match_time', 'referee', 'notes', 'replay_x', 'match_time_x', 'stadium_id_x', 'host_country_x', 'home_team_code_x', 'away_team_code_x', 'score_x', 'away_team_win_x', 'tournament_id_y', 'tournament_name_y', 'd7_match_id_y', 'match_name_y', 'group_stage_y', 'knockout_stage_y',

In [56]:
# Remove duplicate columns, keep first occurrence
for col in ["tournament_id", "tournament_name", "match_name"]:
    idxs = [i for i, c in enumerate(df.columns) if c == col]
    if len(idxs) > 1:
        drop_idxs = set(idxs[1:])
        keep_mask = [i not in drop_idxs for i in range(len(df.columns))]
        df = df.loc[:, keep_mask]

In [57]:
# Drop columns
cols_to_drop = [
    "stage_norm",
    "group_name_d7new",
    "group_stage",
    "knockout_stage",
    "replayed",
    "replay",
    "home_team_norm",
    "away_team_norm",
]
for col in cols_to_drop:
    if col in df.columns:
        df = df.drop(columns=[col])

In [58]:
# Convert draw/extra_time/penalty_shootout values 0->no, 1->yes
for col in ["draw", "extra_time", "penalty_shootout"]:
    if col in df.columns:
        df[col] = df[col].map({0: "no", 1: "yes"}).fillna(df[col])

In [59]:
# Drop home_penalties, away_penalties, home_xg, away_xg
for col in ["home_penalties", "away_penalties", "home_xg", "away_xg"]:
    if col in df.columns:
        df = df.drop(columns=[col])

In [60]:
# Delete second stadium_name column
if "stadium_name_y" in df.columns:
    df = df.drop(columns=["stadium_name_y"])

# handle duplicate stadium_name columns if still present
idxs = [i for i, c in enumerate(df.columns) if c == "stadium_name"]
if len(idxs) > 1:
    drop_idxs = set(idxs[1:])
    keep_mask = [i not in drop_idxs for i in range(len(df.columns))]
    df = df.loc[:, keep_mask]

In [ ]:
# Drop columns
cols_to_drop = [
    "city_name",
    "replay_x",
    "match_time_x",
    "stadium_id_x",
    "host_country_x",
    "away_team_win_x",
]
for col in cols_to_drop:
    if col in df.columns:
        df = df.drop(columns=[col])

In [62]:
# Rename *_x columns to base names (fill target if exists)
rename_pairs = {
    "home_team_code_x": "home_team_code",
    "away_team_code_x": "away_team_code",
    "score_x": "score",
}
for src, tgt in rename_pairs.items():
    if src in df.columns:
        if tgt in df.columns:
            df[tgt] = df[tgt].fillna(df[src])
            df = df.drop(columns=[src])
        else:
            df = df.rename(columns={src: tgt})

In [63]:
# Drop *_y columns and d7_match_id, then resave cleaned CSV
cols_to_drop = [
    "tournament_id_y",
    "tournament_name_y",
    "d7_match_id_y",
    "match_name_y",
    "group_stage_y",
    "knockout_stage_y",
    "replayed_y",
    "replay_y",
    "match_time_y",
    "stadium_id_y",
    "city_name_y",
    "host_country_y",
    "home_team_id_y",
    "home_team_code_y",
    "away_team_id_y",
    "away_team_code_y",
    "score_y",
    "home_score_margin_y",
    "away_score_margin_y",
    "home_team_win_y",
    "away_team_win_y",
    "draw_y",
    "d7_match_id",
]
for col in cols_to_drop:
    if col in df.columns:
        df = df.drop(columns=[col])


In [64]:
# Drop columns
for col in ["home_score_margin", "away_score_margin", "home_team_id", "away_team_id", "group_name"]:
    if col in df.columns:
        df = df.drop(columns=[col])

In [65]:
# Fill missing tournament_id and tournament_name using year
if "tournament_id" in df.columns and "year" in df.columns:
    df["tournament_id"] = df["tournament_id"].fillna("WC-" + df["year"].astype(str))

if "tournament_name" in df.columns and "year" in df.columns:
    df["tournament_name"] = df["tournament_name"].fillna(df["year"].astype(str) + " FIFA World Cup")

In [66]:
# Drop rows with missing home_goals, away_goals, or score (only before 2026)
required_cols = ["home_goals", "away_goals", "score", "year"]
existing_required = [c for c in required_cols if c in df.columns]
if set(["home_goals", "away_goals", "score", "year"]).issubset(df.columns):
    mask_before_2026 = df["year"] < 2026
    df = df[~(mask_before_2026 & df[["home_goals", "away_goals", "score"]].isna().any(axis=1))]
else:
    # fallback: only drop if required columns exist but year is missing
    if all(c in df.columns for c in ["home_goals", "away_goals", "score"]):
        df = df.dropna(subset=["home_goals", "away_goals", "score"])

In [67]:
df

,year,date,tournament_id,tournament_name,match_name,stage,home_team,away_team,home_team_code,away_team_code,...,stadium,stadium_id,stadium_name,city,host_country,host_team,attendance,match_time,referee,notes
0,1930,1930-07-13,WC-1930,1930 FIFA World Cup,France v Mexico,Group stage,France,Mexico,FRA,MEX,...,Estadio Pocitos,S-193,Estadio Pocitos,Montevideo,Uruguay,0.0,4444,15:00,Domingo Lombardi,NaN
1,1930,1930-07-13,WC-1930,1930 FIFA World Cup,United States v Belgium,Group stage,United States,Belgium,USA,BEL,...,Estadio Gran Parque Central,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,0.0,18346,15:00,Jose Macias,NaN
2,1930,1930-07-14,WC-1930,1930 FIFA World Cup,Romania v Peru,Group stage,Romania,Peru,ROU,PER,...,Estadio Pocitos,S-193,Estadio Pocitos,Montevideo,Uruguay,0.0,2549,14:50,Alberto Warnken,NaN
3,1930,1930-07-14,WC-1930,1930 FIFA World Cup,Yugoslavia v Brazil,Group stage,Yugoslavia,Brazil,YUG,BRA,...,Estadio Gran Parque Central,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,0.0,24059,12:45,Anibal Tejada,NaN
4,1930,1930-07-15,WC-1930,1930 FIFA World Cup,Argentina v France,Group stage,Argentina,France,ARG,FRA,...,Estadio Gran Parque Central,S-192,Estadio Gran Parque Central,Montevideo,Uruguay,0.0,23409,16:00,Gilberto Rego,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,2026,2026-07-12,WC-2026,2026 FIFA World Cup,NaN,Quarterfinals,NaN,NaN,NaN,NaN,...,Arrowhead Stadium,NaN,NaN,Kansas City,USA,NaN,<NA>,NaN,NaN,NaN
1070,2026,2026-07-14,WC-2026,2026 FIFA World Cup,NaN,Semifinals,NaN,NaN,NaN,NaN,...,AT&T Stadium,NaN,NaN,Dallas,USA,NaN,<NA>,NaN,NaN,NaN
1071,2026,2026-07-15,WC-2026,2026 FIFA World Cup,NaN,Semifinals,NaN,NaN,NaN,NaN,...,Mercedes-Benz Stadium,NaN,NaN,Atlanta,USA,NaN,<NA>,NaN,NaN,NaN
1072,2026,2026-07-18,WC-2026,2026 FIFA World Cup,NaN,Third Place Playoff,NaN,NaN,NaN,NaN,...,Hard Rock Stadium,NaN,NaN,Miami,USA,NaN,<NA>,NaN,NaN,NaN


In [68]:
# Save cleaned dataset to CSV
output_path = Path("..") / "data" / "cleaned_project_dataset.csv"
df.to_csv(output_path, index=False)
print("Saved cleaned dataset to:", output_path)

Saved cleaned dataset to: ../data/cleaned_project_dataset.csv
